In [1]:
import pandas as pd
import numpy as np
from datetime import date
import sys
sys.path.append("../")
import src.soporte as sp
import warnings
warnings.filterwarnings("ignore")

c:\Users\Lin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Loading my scraped data
dia_18 = pd.read_csv("../data/dia_2023-01-18.csv", index_col = 0)
dia_19 = pd.read_csv("../data/dia_2023-01-19.csv", index_col = 0)
dia_20 = pd.read_csv("../data/dia_2023-01-20.csv", index_col = 0)
dia_30 = pd.read_csv("../data/dia_2023-01-30.csv", index_col = 0)
dia_02 = pd.read_csv("../data/dia_2023-02-04.csv", index_col = 0)
mercadona_20 = pd.read_csv("../data/merc2023-01-20_suma.csv", index_col = 0)

In [3]:
# Loading my data
mercadona = pd.read_csv("../data/mercadona_limpio.csv", index_col = 0)
dia = pd.read_csv("../data/dia_limpio.csv", index_col = 0)

In [4]:
# Changin names and dropping the old category
mercadona.drop(["category"], axis = 1, inplace = True)
mercadona.rename(columns = {"categoria": "category"}, inplace = True)

In [5]:
# Making small ajustment
mercadona_20.drop(["Unnamed: 0.1", "Unnamed: 0", "category_id"],axis= 1, inplace = True) # Droping some error from the pass
mercadona_20.drop_duplicates(inplace = True) # Droping useless info
mercadona_20["subcategoria"] = mercadona_20.apply(lambda x: sp.mer_subcat(x["category"]), axis = 1) # Making the subcategory
mercadona_20["category"] = mercadona_20.apply(lambda x: sp.category(x["subcategoria"]), axis = 1) # Making the category
mercadona_20["supermarket"] = "mercadona-es" # Adding the "supermarket" field

In [6]:
# Droping Nulls and adding the field "supermarket" to my scraped data
dia_18.dropna(inplace= True)
dia_18["supermarket"] = "dia-es"
dia_19.dropna(inplace= True)
dia_19["supermarket"] = "dia-es"
dia_20.dropna(inplace= True)
dia_20["supermarket"] = "dia-es"
dia_30.dropna(inplace= True)
dia_30["supermarket"] = "dia-es"
dia_02.dropna(inplace= True)
dia_02["supermarket"] = "dia-es"

In [7]:
# Here I'm grouping all my data
mercadona_total = pd.concat([mercadona, mercadona_20], axis = 0, ignore_index = True)
dia_sumando = pd.concat([dia, dia_18], axis = 0, ignore_index = True)
dia_sumando = pd.concat([dia_sumando, dia_19], axis = 0, ignore_index = True)
dia_sumando = pd.concat([dia_sumando, dia_20], axis = 0, ignore_index = True)
dia_sumando = pd.concat([dia_sumando, dia_30], axis = 0, ignore_index = True)
dia_total = pd.concat([dia_sumando, dia_02], axis = 0, ignore_index = True)
dia_mercadona_total = pd.concat([mercadona_total, dia_total], axis = 0, ignore_index = True)

In [8]:
# Saving it for later usage
today = date.today()
dia_mercadona_total.to_csv(f"../data/dia_merc_todo{today}.csv")

In [9]:
# Spliting my data
dia_todo = dia_mercadona_total[dia_mercadona_total["supermarket"] == "dia-es"]
mercadona_todo = dia_mercadona_total[dia_mercadona_total["supermarket"] == "mercadona-es"]
mercadona_todo.head(2)

In [10]:
# Filtering so I have the info of the categories that I'm interested in
dia_filtrado = dia_todo[dia_todo["category"].isin(["carniceria", "pescaderia", "verduleria"])]
mercadona_filtrado = mercadona_todo[mercadona_todo["category"].isin(["carniceria", "pescaderia", "verduleria"])]
mercadona_filtrado.head(2)

In [11]:
# Chaging the dtype to datetime
dia_filtrado["insert_date"] = pd.to_datetime(dia_filtrado["insert_date"])
mercadona_filtrado["insert_date"] = pd.to_datetime(mercadona_filtrado["insert_date"])
mercadona_filtrado.head(2)

In [12]:
# Droping the day, so I have the info by Year and Moth
dia_filtrado["insert_date"] = dia_filtrado["insert_date"].dt.strftime("%Y-%m")
mercadona_filtrado["insert_date"] = mercadona_filtrado["insert_date"].dt.strftime("%Y-%m")
mercadona_filtrado.head(2)

In [13]:
# Grouping bu date and name, and making the mean of the prices
dia_agrupado = dia_filtrado.groupby(["insert_date", "name"]).agg([np.mean]).reset_index()
merc_agrupado = mercadona_filtrado.groupby(["insert_date", "name"]).agg([np.mean]).reset_index()
merc_agrupado.head(2)

In [14]:
# Changing the name of the columns
dia_agrupado.columns = "insert_date", "name", "avg_price", "avg_reference_price"
merc_agrupado.columns = "insert_date", "name", "avg_price", "avg_reference_price"
merc_agrupado.head(2)

In [15]:
# Droping the columns of the old data, so I can recover the info of the categories and the subcategories
dia_nombres = dia_filtrado.drop(["insert_date", "price", "reference_price"], axis =1)
mer_nombres = mercadona_filtrado.drop(["insert_date", "price", "reference_price"], axis =1)
mer_nombres.head(2)

In [16]:
# Droping duplicates, so there is only one row with the info that I need
dia_nombres = dia_nombres.drop_duplicates(subset='name', keep="first")
mer_nombres = mer_nombres.drop_duplicates(subset='name', keep="first")
mer_nombres.head(2)

In [21]:
# Merging my info so I have all the info that I need
dia_bien = dia_agrupado.merge(dia_nombres, how = "left", on = "name")
mer_bien = merc_agrupado.merge(mer_nombres, how = "left", on = "name")
mer_bien.head(2)

In [24]:
# Joining the info into one dataframe
mer_dia_bien = pd.concat([dia_bien, mer_bien], axis = 0, join = "outer", ignore_index = True)
mer_dia_bien.head(2)

In [25]:
# Saving everything to use it
today = date.today()
mer_dia_bien.to_csv(f"../data/dia_merc_bien{today}.csv")